In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Set device (GPU if available, else CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [2]:
def differentiable_tmm_normal(n_layers, d_layers, wavelengths):
    """
    A PyTorch implementation of the Transfer Matrix Method for normal incidence.
    Allows backpropagation through refractive indices (n_layers) and thicknesses (d_layers).
    """
    # Constants
    num_wavelengths = wavelengths.shape[0]
    num_layers = n_layers.shape[0]

    # Ensure inputs are complex for TMM calculations
    n_layers_c = n_layers.unsqueeze(0).expand(num_wavelengths, -1).cfloat()
    d_layers_c = d_layers.unsqueeze(0).expand(num_wavelengths, -1).cfloat()
    wl_c = wavelengths.unsqueeze(1).cfloat()

    # Wave vector in each layer: k = 2 * pi * n / lambda
    k = 2 * np.pi * n_layers_c / wl_c

    # Phase shift for each layer: phi = k * d
    # Note: First and last layers are semi-infinite air/substrate, so d=0 effectively for phase,
    # but we handle them as boundaries below.
    phi = k * d_layers_c

    # 2x2 Transfer matrices for each layer j
    # M_j = [[cos(phi), (-i/n)*sin(phi)], [-i*n*sin(phi), cos(phi)]]
    cos_phi = torch.cos(phi)
    sin_phi = torch.sin(phi)
    zeros = torch.zeros_like(cos_phi)
    ones = torch.ones_like(cos_phi)

    # Stack components into (Num_WL, Num_Layers, 2, 2) tensor
    M = torch.stack([
        torch.stack([cos_phi, -1j / n_layers_c * sin_phi], dim=-1),
        torch.stack([-1j * n_layers_c * sin_phi, cos_phi], dim=-1)
    ], dim=-2)

    # Compute total transfer matrix by multiplying all layer matrices
    # Start with Identity matrix
    M_total = torch.eye(2, dtype=torch.cfloat, device=DEVICE).unsqueeze(0).repeat(num_wavelengths, 1, 1)

    # Loop through standard layers (excluding semi-infinite substrate/air if handled separately,
    # here we assume d=0 for first/last if they are just boundaries).
    # For a standard stack: Air -> L1 -> L2 ... -> LN -> Substrate
    for i in range(num_layers):
         M_total = torch.matmul(M_total, M[:, i, :, :])

    # Calculate Transmission Coefficient t
    # t = 2 * n0 / (M00 + M01*ns + n0*M10 + n0*M11*ns)
    # Assuming n0 (air) is index 0, and ns (substrate) is last index.
    n0 = n_layers_c[:, 0]
    ns = n_layers_c[:, -1]

    m00 = M_total[:, 0, 0]
    m01 = M_total[:, 0, 1]
    m10 = M_total[:, 1, 0]
    m11 = M_total[:, 1, 1]

    t = (2 * n0) / (m00 + m01 * ns + n0 * m10 + n0 * m11 * ns)

    # Transmittance T = |t|^2 * (ns / n0)
    T = torch.abs(t)**2 * (torch.real(ns) / torch.real(n0))

    return T

In [3]:
class OnlineOptimizer(nn.Module):
    def __init__(self, num_layers_to_optimize, seed_size=100):
        super(OnlineOptimizer, self).__init__()
        # Simple MLP architecture as suggested by general deep learning practices in standard inverse design
        self.net = nn.Sequential(
            nn.Linear(seed_size, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, num_layers_to_optimize),
            nn.Sigmoid() # Sigmoid to bound output between 0 and 1 for easy scaling
        )

    def forward(self, seed):
        # Output is in [0, 1]. We scale this to the desired refractive index range.
        # The text specifies a range of [1.6, 2.4] for SiNx[cite: 1188].
        n_min = 1.6
        n_max = 2.4
        out = self.net(seed)
        n_optimized = out * (n_max - n_min) + n_min
        return n_optimized

In [4]:
def apply_systematic_error(n_layers, error_type='linear_gradient'):
    """Applies deterministic errors as described in Section 3.4.3"""
    num_active_layers = n_layers.shape[0]
    device = n_layers.device

    if error_type == 'linear_gradient':
        # Simulate a linear drift in refractive index from bottom to top [cite: 1370-1371]
        # Max drift 0.4 as per text example [cite: 1371]
        drift = torch.linspace(0, 0.4, steps=num_active_layers, device=device)
        return n_layers + drift

    # Add other types (sinusoidal, transition regions) here as needed
    return n_layers

def apply_random_error(n_layers, gamma=0.01, mu=0.0):
    """
    Applies random additive noise as described in Equation 3.6[cite: 1514].
    n_imp = n_id + n_id * n_rand
    n_rand = 2 * gamma * U(0,1) - mu
    """
    # Uniform noise U(0,1)
    U = torch.rand_like(n_layers)
    n_rand = 2 * gamma * U - mu
    n_imperfect = n_layers + (n_layers * n_rand)
    return n_imperfect

In [5]:
# --- Setup Parameters ---
NUM_LAYERS = 50  # Number of layers to optimize (excluding air/substrate)
SEED_SIZE = NUM_LAYERS # Size of the random seed vector
WAVELENGTHS = torch.linspace(400, 700, 300).to(DEVICE) # Visible spectrum 400-700nm [cite: 1097]
TARGET_SPECTRUM = torch.ones_like(WAVELENGTHS)
# Define a simple Bandstop target (e.g., reflect 530-570nm)
TARGET_SPECTRUM[(WAVELENGTHS > 530) & (WAVELENGTHS < 570)] = 0.0

# Fixed parameters
N_AIR = torch.tensor([1.0]).to(DEVICE)
N_SUBSTRATE = torch.tensor([1.5]).to(DEVICE) # Approx glass/SiO2
LAYER_THICKNESS = 30e-9 # 30nm fixed thickness as in some examples [cite: 1186]

# --- Initialization ---
# The fixed seed that remains constant throughout optimization [cite: 1179]
FIXED_SEED = torch.rand(SEED_SIZE).to(DEVICE)

model = OnlineOptimizer(NUM_LAYERS, SEED_SIZE).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3) # Adam is standard [cite: 2088]
criterion = nn.MSELoss()

# --- Training Helper Function ---
def train_step(stage_2=False, error_type=None, rand_gamma=0.0):
    optimizer.zero_grad()

    # 1. NN generates "ideal" design from fixed seed
    n_generated = model(FIXED_SEED)

    # 2. Apply Fabrication Errors if in Stage 2 [cite: 1482]
    if stage_2:
        if error_type:
             n_used = apply_systematic_error(n_generated, error_type)
        else:
             n_used = n_generated

        if rand_gamma > 0:
             n_used = apply_random_error(n_used, gamma=rand_gamma)
    else:
        n_used = n_generated

    # Assemble full stack for TMM (Air + Active Layers + Substrate)
    n_full_stack = torch.cat([N_AIR, n_used, N_SUBSTRATE])

    # Create thickness tensor (0 for air/substrate to treat as infinite boundaries in this simple TMM)
    d_active = torch.full((NUM_LAYERS,), LAYER_THICKNESS, device=DEVICE)
    d_full_stack = torch.cat([torch.tensor([0.0], device=DEVICE), d_active, torch.tensor([0.0], device=DEVICE)])

    # 3. Differentiable TMM Solver calculates spectrum
    transmission = differentiable_tmm_normal(n_full_stack, d_full_stack, WAVELENGTHS * 1e-9)

    # 4. Calculate Loss & Backpropagate
    loss = criterion(transmission, TARGET_SPECTRUM)
    loss.backward()
    optimizer.step()

    return loss.item(), transmission.detach(), n_generated.detach()


# --- Execution ---

# STAGE 1: Ideal Optimization [cite: 1184]
print("Starting Stage 1: Ideal Optimization...")
for epoch in range(500):
    loss_val, current_spectrum, current_design = train_step(stage_2=False)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss_val:.6f}")

# STAGE 2: Fabrication-in-the-loop Retraining [cite: 1468]
# We continue training the SAME model, but now with errors engaged.
print("\nStarting Stage 2: Fabrication-in-the-loop Retraining (Linear Drift)...")
for epoch in range(501, 801):
    # Applying a linear gradient systematic error during training
    loss_val, current_spectrum, final_design = train_step(stage_2=True, error_type='linear_gradient')
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss_val:.6f}")

print("Optimization Complete.")

# Final check: The NN output 'final_design' is now "pre-distorted" to
# compensate for the linear gradient error.

Starting Stage 1: Ideal Optimization...
Epoch 0, Loss: 0.124598
Epoch 100, Loss: 0.031449
Epoch 200, Loss: 0.030015
Epoch 300, Loss: 0.029473
Epoch 400, Loss: 0.029348

Starting Stage 2: Fabrication-in-the-loop Retraining (Linear Drift)...
Epoch 600, Loss: 0.026823
Epoch 700, Loss: 0.026763
Epoch 800, Loss: 0.026730
Optimization Complete.
